In [25]:
import os
import re
import json
import chardet
import sqlite3
import public_function as pf
from tencentfanyi import translate as tencent_t
from baidufanyi import translate as baidu_t

# NOTE 中间的所有文件都是UTF8
encoding = 'cp932'
output_encoding = 'utf8'
name = '[190830][ANIM.teamMM] 妻の肉穴にホームステイするマッチョ留学生2軒目～元巨乳グラドルの妻は、ダニーズ・ブートキャンプで汁だくメス子宮に開発されていた～'

file_all = os.listdir()
if 'intermediate_file' not in file_all:
    os.mkdir('intermediate_file')
if 'output' not in file_all:
    os.mkdir('output')
if 'input' not in file_all:
    os.mkdir('input')

In [ ]:
# FIXME 从游戏原文本文件提取日文

def has_jp(line: str) -> bool:
    '''
    如果含有日文文字（除日文标点）则认为传入文字含有日文, 返回true
    '''
    for ch in line:
        if ch >= '\u0800' and ch < '\u4e00':
            return True
    return False


def get_scenario_from_origin(data: list) -> list:
    text_all = []
    cnt = 0
    # START FIXME
    for line in data:
        if pf.has_jp(line) and line[:11] == '%SetMessage':
            line = re.sub(r'/.','',line[14:-20])
            text_all.append(line+'\n')
        cnt+=1
    # END
    return text_all

pf.extract_jp(get_scenario_from_origin, encoding)


In [ ]:
# FIXME 创建翻译字典
pf.create_dict(lambda x:x[:-1])

In [ ]:
# 检查字典key数量是否正确
pf.check_dict()

In [ ]:
# 把字典翻译

def delete_func(text:str):
    '''
    \[.*?\]|<.+?>|,|\\n|　|―+|\\|"|\]|\[|\/|\;|[a-z0-9A-Z]
    '''
    patt = r'\[.*?\]|<.+?>|\|'
    return re.sub(patt, '', text)

# delete_func('miz驚き赤み6')
pf.translate(delete_func=None)
# print(pf._translate('しかし、時に思うのだ。あの毒花のような女に誘われた時、それを拒絶できていたなら、違う|現在[いま]になっていただろうか、と。', delete_func=delete_func))

In [ ]:
# 检查是否还有没翻译的语句
pf.check_dict_untranslated()

In [ ]:
# 构建对比
pf.create_contrast()

In [ ]:

def find_all(data:list, failed_text:list, jp_chs:dict):
    record = 0
    cnt = 0                         # 当前行
    start = False
    for line in data:
        #START FIXME 替换目标文本
        if pf.has_jp(line) and line[0] not in ('{','n'):
            key = line[1:-3]
            if key in jp_chs:
                record += 1
                data[cnt] = data[cnt].replace(key, jp_chs[key])
            else:
                failed_text.append(key+'\n')
        cnt += 1
        # END 
    return record

pf.replace_all(find_all, encoding, output_encoding)

In [ ]:
'僇僗僞儉僀儞僗僩乕儖帪偼僨傿僗僋儗僗偱偺婲摦偼偱偒傑偣傫丅'.encode('gbk').decode('cp932')

In [ ]:
b'\x81\x76'.decode('cp932')

In [ ]:
'你好'.encode('gbk')

In [ ]:
'「イク、イクッ、イクイククッ、イギュッ！イックぅううううううっ▼▼▼」'.find('「イク、イクッ、イクイククッ、イギュッ！')

In [ ]:

def create_key():
    Key = b'\xAA'*12
    Key = bytearray(Key)
    Key[0] ^= 0xff
    Key[1] = (( Key[1] >> 4 ) | ( Key[1] << 4 ) )&0xff
    Key[2] = Key[2] ^ 0x8a 
    Key[3] = ((( ( Key[3] >> 4 ) | ( Key[3] << 4 ) )) ^ 0xff) & 0xff
    Key[4] ^= 0xff
    Key[5] = Key[5] ^ 0xac 
    Key[6] ^= 0xff
    Key[7] = ( ((( Key[7] >> 3 ) | ( Key[7] << 5 ) ) )^0xff )& 0xff
    Key[8] = (( Key[8] >> 5 ) | ( Key[8] << 3 ))&0xff 
    Key[9] = Key[9] ^ 0x7f 
    Key[10] = (( ( Key[10] >> 4 ) | ( Key[10] << 4 ) ) ^ 0xd6 )& 0xff
    Key[11] = Key[11] ^ 0xcc 
    return Key
list(map(lambda x:print(hex(x)), Key))

In [ ]:
raw = b'\xCE\xDF\xE8\xD6\xDF\xEC\xDF\xEC\xE0\xE4'
ans = ''
for i in raw:
    ans += hex(i-0x2d)[2:] + ' '
print(ans)

In [ ]:
b'\x0f\x01M\x01\x8b\x01\xc9\x01\x07\x02E\x02\x83\x02\xc1\x02\xff\x02=\x03{\x03\xb9\x03\xf7\x035\x04y\x04;;'.decode('cp932', errors='ignore')

In [ ]:
def dec(a:int, b:int):
    return (a | b) & (~(a & b) & 0xff)
print(hex(dec(0x54, 0x1f)))

In [ ]:
'乽'.encode('gbk').decode('cp932')

In [ ]:
import re
s = '出来上がった叶愛@[謹製:きんせい]のハンバーグカレーをテーブルへと運んだ。'
a = re.findall(r'@\[.*?\]',s)
print(re.sub(r'@\[.*?\]','',s))
print(a)

In [ ]:
print(re.findall)